In [ ]:
# test push
# Imports
from PIL import Image
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Converting the image to a numpy array
def img_to_arr(the_path):
    img = Image.open(the_path)
    img = img.convert("L")
    data = list(img.getdata())
    data = np.array(data)

    return data

In [ ]:
img_path = "./img.jpg" # Image source path

In [ ]:
img = Image.open(img_path)
print(img.format, img.size, img.mode)

In [ ]:
img_arr = img_to_arr(img_path)
print(img_arr)
print(len(img_arr))

In [ ]:
# Create a zero array with 256 length to store the count of the pixel values.
pixel_counts = np.zeros(256, dtype=int)

unique_values, counts = np.unique(img_arr, return_counts=True)

print("Unique Values : \n",unique_values)
print("Count Of Each Value : \n",counts)

pixel_counts[unique_values] = counts

print("Pixel Counts: \n", pixel_counts)

In [ ]:
# Visualize the pixel count with a histogram.
x_values = np.arange(len(pixel_counts))

# Creating a bar plot
plt.figure(figsize=(35,20))
plt.bar(x_values, pixel_counts, color='blue', alpha=0.7)
plt.xlabel('Pixel Color')
plt.ylabel('Frequency')
plt.title('Pixel Color Frequency Histogram')

plt.show()

In [ ]:
# This is the indices of pixel_counts array if it is sorted
pixel_values_sorted = np.argsort(pixel_counts)
print(pixel_values_sorted)

In [ ]:
# Create an array to calcualte the slope. This is for right side
def create_arr_for_slope_r(arr, start, limit):
    end_index = 0
    if (start + limit > 255):
        end_index = 255
    else:
        end_index = start + limit
    
    selected_x = np.arange(start, end_index, dtype=int)
    selected_y = arr[start:end_index]

    return selected_x, selected_y

In [ ]:
# Create an array to calcualte the slope. This is for right side
def create_arr_for_slope_l(arr, start, limit):
    end_index = 0
    if (start - limit < 0):
        end_index = 0
    else:
        end_index = start - limit

    selected_x = np.arange(end_index, start, dtype=int)
    selected_y = arr[end_index:start]

    return selected_x, selected_y

In [ ]:
# This function trys to find the threshold to be the segmentation point and the spawning area around the threshold.
def check_if_threshold_true(pixel_threshhold_arr, pixel_amount_arr):
    spawn = 16 # Arbitrary spawn start value
    flag = 0
    results = []

    while (flag != -1):
        if(flag == 128): # Because there is 256 different potential threshold points we choose to only look for half of them
            break
        
        print("LOOP : ### ", flag)
        print("SPAWN : ", spawn)
        print("RESULTS : ", results)
        print("\n\n\n")

        # Create the arrays to calculate the slope
        arr_x_left, arr_y_left = create_arr_for_slope_l(pixel_amount_arr, pixel_threshhold_arr[flag], spawn)
        arr_x_rigth, arr_y_rigth = create_arr_for_slope_r(pixel_amount_arr, pixel_threshhold_arr[flag], spawn)

        # Calculate the slope
        left_slope, _ = np.polyfit(arr_x_left, arr_y_left, 1)
        rigth_slope, _ = np.polyfit(arr_x_rigth, arr_y_rigth, 1)

        print("L SLOPE : ", left_slope)
        print("R SLOPE : ", rigth_slope)
        print("\n")
        
        # We want the left slope to be negative and right slope to be positive if any of this is false we are on wrong threshold 
        if (left_slope > 1 or rigth_slope < -1):
            flag += 1
            spawn = 16
            print("IF 1 --- l_slope, r_slope", left_slope, rigth_slope)
            print("IF 1 --- flag, spawn", flag, spawn)
            continue
        
        # If the slopes are not steep enough we increase the spawning area around the threshold. This prevents finding thresholds side by side. We want thresholds to be discrete.
        if (abs(left_slope) < 5 or rigth_slope < 5):
            spawn += 8
            print("IF 2 --- l_slope, r_slope", left_slope, rigth_slope)
            print("IF 2 --- flag, spawn", flag, spawn)
            print("\n\n\n")
            continue
        
        temp = {"pixel_value" : pixel_threshhold_arr[flag], "spawn" : spawn}

        # Check if the found threshold is in another threshold's spawn area. If not add it to the results
        is_in_range = any(record['pixel_value'] - record['spawn'] <= temp['pixel_value'] <= record['pixel_value'] + record['spawn'] for record in results)

        if not is_in_range:
            results.append(temp)
            flag += 1
            spawn = 16
            
            print("New record added successfully!")
        else:
            flag += 1
            spawn = 16
            print("Record already exists within the range in the list.")


    return results

In [ ]:
result_thresholds = check_if_threshold_true(pixel_values_sorted, pixel_counts)

In [ ]:
result_thresholds

In [ ]:
# Sort result thresholds
result_thresholds = sorted(result_thresholds, key=lambda x: x['pixel_value'])

In [ ]:
# This function creates a PNG image with alpha channel so unwanted parts can be set to transparent.
def create_segmented_img(orj_img, orj_img_arr, lower_limit, upper_limit, section):
    
    # To construct a new image we give original image's size
    width = orj_img.size[0]
    height = orj_img.size[1]
    new_image = Image.new('RGBA', (width, height))
    
    # We iterate over all pixels and if the pixel value is what we wanted put the pixel as it is if not make it transparent.
    for y in range(height):
        for x in range(width):
            pixel_value = orj_img_arr[y * width + x]
            if (lower_limit <= pixel_value <= upper_limit):
                new_image.putpixel((x, y), (pixel_value, pixel_value, pixel_value))
            else:
                new_image.putpixel((x, y), (0, 0, 0, 0))

    # Save the image
    new_image.save('image_segment_'+ section +'.png', format='PNG')


In [ ]:
lower_boundary = 0
upper_boundary = 0

# We iterate over thresholds and give the segment create function needed limits
for i in range(len(result_thresholds) + 1):

    upper_boundary = result_thresholds[i]["pixel_value"]
    print("UPPER : ", upper_boundary)
    print("LOWER : ", lower_boundary)
    create_segmented_img(img, img_arr, lower_boundary, upper_boundary, str(i+1))

    if (i + 1 == len(result_thresholds)):
        upper_boundary = 255
        lower_boundary = result_thresholds[i]["pixel_value"]
        print("UPPER FNL : ", upper_boundary)
        print("LOWER FNL: ", lower_boundary)
        create_segmented_img(img, img_arr, lower_boundary, upper_boundary, str(i+2))
        break

    lower_boundary = result_thresholds[i]["pixel_value"]
    upper_boundary = result_thresholds[i+1]["pixel_value"]
    print("UPPER UPT : ", upper_boundary)
    print("LOWER UPT: ", lower_boundary)

